# **2. Data Preprocessing**

## **2.1. Mục tiêu**
* Xử lý giá trị thiếu
* Loại bỏ các giá trị ngoại lai
* Loại bỏ các cột không cần thiết
* Tạo các đặc trưng mới
* Chuẩn bị dữ liệu cho các mô hình

## **2.2. Import thư viện & Load dữ liệu**

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('hotel_bookings.csv')
df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


## **2.3. Các bước làm sạch dữ liệu**

### **2.3.1. Phân loại**

In [10]:
# -----------------------------
# 1. Xử lý missing values
# ---------------------------
df_classification = df.copy()
df_classification.drop_duplicates(inplace=True)  # Loại bỏ giá trị trùng

df_classification['children'] = df_classification['children'].fillna(df_classification['children'].median()) # Thay thế bằng median
df_classification['babies'] = df_classification['babies'].fillna(df_classification['babies'].median()) # Thay thế bằng median
df_classification['country'] = df_classification['country'].fillna(df_classification['country'].mode()[0]) # Thay thế bằng Median
df_classification['agent'] = df_classification['agent'].fillna(0) # Thay thế bằng 0
df_classification['company'] = df_classification['company'].fillna(0) # Thay thế bằng 0

# -----------------------------
# 2. Loại bỏ dữ liệu không hợp lệ
# -----------------------------
df_classification = df_classification[(df_classification['adults'] + df_classification['children'] + df_classification['babies']) > 0]
df_classification = df_classification[df_classification['adr'] >= 0]  # bỏ âm

# -----------------------------
# 3. Feature Engineering
# -----------------------------

# Tổng số ngày lưu trú / tổng số khách
df_classification['total_stay'] = df_classification['stays_in_week_nights'] + df_classification['stays_in_weekend_nights']
df_classification['total_guests'] = df_classification['adults'] + df_classification['children'] + df_classification['babies']

# Các biến dự báo mạnh
df_classification['is_family_trip'] = (df_classification['children'] + df_classification['babies'] > 0).astype(int)
df_classification['has_previous_cancel'] = (df_classification['previous_cancellations'] > 0).astype(int)
df_classification['adr_per_guest'] = df_classification['adr'] / df_classification['total_guests']

# Chuyển tháng sang số
df_classification['arrival_month_num'] = pd.to_datetime(df['arrival_date_month'], format='%B').dt.month

# Tạo biến season
def get_season(month):
    if month in [12,1,2]: return 'Winter'
    if month in [3,4,5]: return 'Spring'
    if month in [6,7,8]: return 'Summer'
    return 'Fall'

df_classification['season'] = df_classification['arrival_month_num'].apply(get_season) # Mùa du lịch

# Tạo biến arrival weekday
df_classification['arrival_date'] = pd.to_datetime(
    df_classification['arrival_date_year'].astype(str) + "-" +
    df_classification['arrival_month_num'].astype(str) + "-" +
    df_classification['arrival_date_day_of_month'].astype(str)
)


df_classification['arrival_weekday'] = df_classification['arrival_date'].dt.weekday # Ngày trong tuần
df_classification.drop(['arrival_date'], axis=1, inplace=True)

# -----------------------------
# 4. Outlier Filtering
# -----------------------------

df_classification['lead_time_log'] = np.log1p(df['lead_time'])
df_classification['adr_log'] = np.log1p(df['adr'])

# -------------------------------
# 5. Loại bỏ cột không cần thiết
# -------------------------------

df_classification.drop(['reservation_status', 'reservation_status_date'], axis=1, inplace=True)

# -----------------------------
# 6. Tách dữ liệu
# -----------------------------
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 1. Tách dữ liệu với tỉ lệ train/test 80 - 29
X = df_classification.drop('is_canceled', axis=1)
y = df_classification['is_canceled']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [11]:
num_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()

print("Numeric:", num_cols) # Xác định cột số
print("Categorical:", cat_cols) # Xác định cột phân loại

Numeric: ['lead_time', 'arrival_date_year', 'arrival_date_week_number', 'arrival_date_day_of_month', 'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'children', 'babies', 'is_repeated_guest', 'previous_cancellations', 'previous_bookings_not_canceled', 'booking_changes', 'agent', 'company', 'days_in_waiting_list', 'adr', 'required_car_parking_spaces', 'total_of_special_requests', 'total_stay', 'total_guests', 'is_family_trip', 'has_previous_cancel', 'adr_per_guest', 'lead_time_log', 'adr_log']
Categorical: ['hotel', 'arrival_date_month', 'meal', 'country', 'market_segment', 'distribution_channel', 'reserved_room_type', 'assigned_room_type', 'deposit_type', 'customer_type', 'season']


In [12]:
df_classification.to_csv('hotel_booking_classification.csv', index=False)

X_train.to_csv('hotel_booking_classification_train.csv', index=False)
y_train.to_csv('hotel_booking_classification_train_target.csv', index=False)
X_test.to_csv('hotel_booking_classification_test.csv', index=False)
y_test.to_csv('hotel_booking_classification_test_target.csv', index=False)

### **2.3.2. Phân cụm**

In [16]:
df_clustering = df.copy()
# ---------------------------
# 1. Xử lý missing values
# ---------------------------
df_clustering['children'] = df_clustering['children'].fillna(df_clustering['children'].median())
df_clustering['babies'] = df_clustering['babies'].fillna(df_clustering['babies'].median())
df_clustering['country'] = df_clustering['country'].fillna('UNK')
df_clustering['agent'] = df_clustering['agent'].fillna(0)
df_clustering['company'] = df_clustering['company'].fillna(0)

# -----------------------------
# 2. Loại bỏ dữ liệu không hợp lệ
# -----------------------------
df_clustering = df_clustering[(df_clustering['adults'] + df_clustering['children'] + df_clustering['babies']) > 0]
df_clustering = df_clustering[df_clustering['adr'] >= 0]

# -----------------------------
# 3. Feature Engineering
# -----------------------------

# Tổng số ngày lưu trú / tổng số khách
df_clustering['total_stay'] = df_clustering['stays_in_week_nights'] + df_clustering['stays_in_weekend_nights']
df_clustering['total_guests'] = df_clustering['adults'] + df_clustering['children'] + df_clustering['babies']

# Các biến dự báo mạnh
df_clustering['is_family_trip'] = (df_clustering['children'] + df_clustering['babies'] > 0).astype(int)
df_clustering['adr_per_guest'] = df_clustering['adr'] / df_clustering['total_guests']

df_clustering['lead_time_log'] = np.log1p(df_clustering['lead_time'])
df_clustering['adr_log'] = np.log1p(df_clustering['adr'])

# -------------------------------
# 4. Loại bỏ cột không cần thiết
# -------------------------------
df_clustering.drop(['reservation_status', 'reservation_status_date'], axis=1, inplace=True)

In [17]:
df_clustering.to_csv('hotel_booking_clustering.csv', index=False)

### **2.3.3. Luật kết hợp**

In [18]:
df_transaction = df.copy()

# ---------------------------
# 1. Xử lý missing values
# ---------------------------
df_transaction['children'] = df_transaction['children'].fillna(0)
df_transaction['babies'] = df_transaction['babies'].fillna(0)
df_transaction['country'] = df_transaction['country'].fillna('UNK')
df_transaction['agent'] = df_transaction['agent'].fillna(0)
df_transaction['company'] = df_transaction['company'].fillna(0)

# -----------------------------
# 2. Loại bỏ dữ liệu không hợp lệ
# -----------------------------
df_transaction = df_transaction[(df_transaction['adults'] + df_transaction['children'] + df_transaction['babies']) > 0]
df_transaction = df_transaction[df_transaction['adr'] >= 0]

# -----------------------------
# 3. Feature Engineering
# -----------------------------
df_transaction['total_stay'] = df_transaction['stays_in_week_nights'] + df_transaction['stays_in_weekend_nights']
df_transaction['is_family_trip'] = (df_transaction['children'] + df_transaction['babies'] > 0).astype(int)

# Phân nhốm lead_time
df_transaction['lead_time_bucket'] = pd.cut(
    df_transaction['lead_time'],
    bins=[-1, 0, 30, 90, 180, 365, df_transaction['lead_time'].max()],
    labels=['0', '1-30','31-90','91-180','181-365','365+']
)

df_transaction['adr_bucket'] = pd.qcut(df_transaction['adr'], q=5, duplicates='drop', labels=['A1','A2','A3','A4','A5'])
df_transaction['stay_bucket'] = pd.cut(df_transaction['total_stay'], bins=[0,1,3,7,14,100], labels=['1','2-3','4-7','8-14','15+'], right=True)

# -----------------------------
# 4. Chọn cột tạo item
# -----------------------------
cols_item = [
    'hotel','meal','market_segment','distribution_channel','customer_type',
    'deposit_type','reserved_room_type','is_repeated_guest',
    'is_family_trip','lead_time_bucket','adr_bucket','stay_bucket',
    'is_canceled'
]

dfr_items = df_transaction[cols_item].astype(str)

# -----------------------------
# 5. Đổi dạng
# -----------------------------
transactions = dfr_items.apply(lambda row: [f"{c}={row[c]}" for c in dfr_items.columns], axis=1)

In [19]:
transactions_df = pd.DataFrame({
    'transaction': transactions.apply(lambda x: ','.join(x))
})

transactions_df.to_csv(
    'hotel_booking_transactions.csv',
    index=False,
    encoding='utf-8'
)

### **2.3.4. Chuỗi thời gian**

In [20]:
df_time_series = df.copy()

# ---------------------------------------
# 1. Xử lý missing values và không hợp lý
# ---------------------------------------
df_time_series['children'] = df_time_series['children'].fillna(0)
df_time_series['babies'] = df_time_series['babies'].fillna(0)
df_time_series['country'] = df_time_series['country'].fillna('UNK')
df_time_series = df_time_series[(df_time_series['adults'] + df_time_series['children'] + df_time_series['babies']) > 0]
df_time_series = df_time_series[df_time_series['adr'] >= 0]
# ---------------------------
# 2. Dựng timeindex
# ---------------------------
df_time_series['arrival_month_num'] = pd.to_datetime(df_time_series['arrival_date_month'], format='%B').dt.month
df_time_series['arrival_date'] = pd.to_datetime(
    df_time_series['arrival_date_year'].astype(str) + "-" +
    df_time_series['arrival_month_num'].astype(str) + "-" +
    df_time_series['arrival_date_day_of_month'].astype(str)
)
df_time_series['total_guests'] = df_time_series['adults'] + df_time_series['children'] + df_time_series['babies']
# ---------------------------
# 3. Lấy các cột đặc trưng thời gian
# ---------------------------
daily = df_time_series.groupby('arrival_date').agg(
    bookings=('hotel','size'),
    canceled=('is_canceled','sum'),
    adr_mean=('adr','mean'),
    lead_time_mean=('lead_time','mean'),
    guests_mean=('total_guests','mean')
).sort_index()

# ---------------------------
# 4. Lắp missing sau khi lấy index
# ---------------------------
daily = daily.asfreq('D')
daily['bookings'] = daily['bookings'].fillna(0)
daily['canceled'] = daily['canceled'].fillna(0)

# dùng quá khứ, tránh leakage
daily['adr_mean'] = daily['adr_mean'].ffill()
daily['lead_time_mean'] = daily['lead_time_mean'].ffill()
daily['guests_mean'] = daily['guests_mean'].ffill()

# ---------------------------
# 5. Tìm đặc trưng cho ML
# ---------------------------
daily['dow'] = daily.index.dayofweek
daily['month'] = daily.index.month
daily['weekofyear'] = daily.index.isocalendar().week.astype(int)

daily['bookings_lag7'] = daily['bookings'].shift(7)
daily['bookings_ma7'] = daily['bookings'].rolling(7).mean()

daily['cancel_rate'] = daily['canceled'] / daily['bookings'].replace(0, np.nan)
daily['cancel_rate'] = daily['cancel_rate'].fillna(0)

daily = daily.dropna()

In [21]:
split_date = daily.index[int(len(daily) * 0.8)]

train = daily[daily.index < split_date]
test  = daily[daily.index >= split_date]

print(train.index.min(), train.index.max())
print(test.index.min(), test.index.max())

2015-07-08 00:00:00 2017-03-26 00:00:00
2017-03-27 00:00:00 2017-08-31 00:00:00


In [22]:
train.to_csv( 'hotel_booking_timeseries_train.csv', index=True)
test.to_csv( 'hotel_booking_timeseries_test.csv', index=True)
